# Get the lever position for every data row of the ses.ap.pose 


The lever position was extracted by DLC running on the `arena_top_cropped.mp4` and the values are stored in `mouseLeverPosition.csv`.
We can use `arena_top.log` to get the ROS time of the frames.

3 body parts were tracked, leading to 6 data points : leverPressX,leverPressY,leverBoxPLX,leverBoxPLY,leverBoxPRX,leverBoxPRY

For making the analysis easier, I will get the position of these 3 body parts at the same time points as the ses.ap.pose matrix. In addition, we will change the reference frame so that the arena center is at 0,0 and that the data are in cm.


In [2]:
%load_ext autoreload
%autoreload 2
%run setup_project.py

/home/kilo/repo/analysis_jingjie/jingjie/autopi_mec_jingjie/autopi_mec/mouse_and_objects_position_tracking/setup_project.py:769: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()
/home/kilo/repo/analysis_jingjie/jingjie/autopi_mec_jingjie/autopi_mec/mouse_and_objects_position_tracking/setup_project.py:797: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()


In [3]:
projectName, dataPath, dlcModelPath, myProject, sSessions= setup_project_session_lists()

creating myProject, an autopipy.project object
Project name: autopi_mec
dataPath: /adata/projects/autopi_mec
dlcModelPath: /adata/models
Reading /adata/projects/autopi_mec/sessionList_invalid
We have 1 testing sessions in the list
spikeA.Kilosort_session objects are in sSessions


In [4]:
prepareSessionsForSpatialAnalysisProject(sSessions,myProject.sessionList)

Loading Animal_pose and Spike_train, sSes.ap and sSes.cg
Will load .pose.npy


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  3.94it/s]

Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


In [5]:
sSes = sSessions[0]
ses = myProject.sessionList[0]
print(sSes.name,sSes.path)

mn8578-30112021-0107 /adata/projects/autopi_mec/mn8578/mn8578-30112021-0107


In [6]:
sSes.ap.pose.shape

(499511, 7)

Set this lever position analysis to run on all our recording sessions

In [9]:
from PIL import Image
def createLeverPoseFile(ses):
    
    if ses.ap.pose.shape[1] != 8:
        raise valueError("Expect animal_pose.pose with 8 columns")
    
    
    
    posLev = pd.read_csv(ses.path+"/"+ses.name+".mouseLeverPosition.csv")
    ti = pd.read_csv(ses.path+"/"+ses.name+".arena_top.log",sep=" ")
    posLev["rosTime"]= ti.time
    
    logRosTimeRange = np.array([np.nanmin(posLev.rosTime),np.nanmax(posLev.rosTime)])
    print("Video ros time: {}, {}".format(np.nanmin(posLev.rosTime),np.nanmax(posLev.rosTime)))
    
    
    arenaImageFile=ses.path+"/arenaDetectionCropped.png"
    im = Image.open(arenaImageFile)
    #plt.imshow(im)
    aCoord = np.loadtxt(ses.path+"/arenaCoordinates")
    #plt.scatter(aCoord[0],aCoord[1])
    #plt.show()
    
    posLev.leverPressX=(posLev.leverPressX-aCoord[0])/aCoord[2]*40
    posLev.leverPressY=(posLev.leverPressY-aCoord[1])/aCoord[2]*40
    posLev.leverBoxPLX=(posLev.leverBoxPLX-aCoord[0])/aCoord[2]*40
    posLev.leverBoxPLY=(posLev.leverBoxPLY-aCoord[1])/aCoord[2]*40
    posLev.leverBoxPRX=(posLev.leverBoxPRX-aCoord[0])/aCoord[2]*40
    posLev.leverBoxPRY=(posLev.leverBoxPRY-aCoord[1])/aCoord[2]*40
    
    # model
    lpx = interp1d(posLev.rosTime,posLev.leverPressX,bounds_error=False)
    lpy = interp1d(posLev.rosTime,posLev.leverPressY,bounds_error=False)
    lblx = interp1d(posLev.rosTime,posLev.leverBoxPLX,bounds_error=False)
    lbly = interp1d(posLev.rosTime,posLev.leverBoxPLY,bounds_error=False)
    lbrx = interp1d(posLev.rosTime,posLev.leverBoxPRX,bounds_error=False)
    lbry = interp1d(posLev.rosTime,posLev.leverBoxPRY,bounds_error=False)

    # interpolate
    apRosTimeRange = np.array([np.nanmin(ses.ap.pose[:,7]),np.nanmax(ses.ap.pose[:,7])])
    print("Animal pose ros time: {}, {}".format(np.nanmin(ses.ap.pose[:,7]),np.nanmax(ses.ap.pose[:,7])))
    
    print("{}, time difference ap, log: {}".format(ses.name,logRosTimeRange-apRosTimeRange))
    
    if logRosTimeRange[0]-apRosTimeRange[0] < 0:
         raise valueError("apRosTimeRange should start before logRosTimeRange")
    if logRosTimeRange[1]-apRosTimeRange[1] > 0 :
         raise valueError("apRosTimeRange should end after logRosTimeRange")
            
    
    
    
    plpx = lpx(ses.ap.pose[:,7])
    plpy = lpy(ses.ap.pose[:,7])
    plblx = lblx(ses.ap.pose[:,7])
    plbly = lbly(ses.ap.pose[:,7])
    plbrx = lbrx(ses.ap.pose[:,7])
    plbry = lbry(ses.ap.pose[:,7])

    leverPose = pd.DataFrame({"time":ses.ap.pose[:,7],
                              "timeRec":ses.ap.pose[:,0],
                          "leverPressX":plpx,
                          "leverPressY":plpy,
                          "leverBoxPLX":plblx,
                          "leverBoxPLY":plbly,
                          "leverBoxPRX":plbrx,
                          "leverBoxPRY":plbry})
    fn = ses.path+"/leverPose"
    print("Saving",fn)
    leverPose.to_csv(fn,index=False)

In [17]:
for sSes in sSessions:
    createLeverPoseFile(sSes)

Video ros time: 1638266042.57488, 1638271678.937942
Animal pose ros time: 1638263295.6051757, 1638273984.242001
mn8578-30112021-0107, time difference ap, log: [ 2746.96970415 -2305.30405903]
Saving /adata/projects/autopi_mec/mn8578/mn8578-30112021-0107/leverPose
Video ros time: 1638712877.486211, 1638718328.179808
Animal pose ros time: 1638709935.1783361, 1638720640.655357
mn8578-05122021-0108, time difference ap, log: [ 2942.30787492 -2312.47554898]
Saving /adata/projects/autopi_mec/mn8578/mn8578-05122021-0108/leverPose
Video ros time: 1638800330.031839, 1638805798.127024
Animal pose ros time: 1638797217.5964122, 1638808190.6468163
mn8578-06122021-0107, time difference ap, log: [ 3112.43542671 -2392.51979232]
Saving /adata/projects/autopi_mec/mn8578/mn8578-06122021-0107/leverPose
Video ros time: 1638878747.197813, 1638884205.7691467
Animal pose ros time: 1638875534.6703386, 1638886553.0403893
mn8578-07122021-0107, time difference ap, log: [ 3212.5274744  -2347.27124262]
Saving /adata/